In [128]:
import os
import numpy as np
import gymnasium as gym
import pandas as pd
import numpy as np
from finrl.main import check_and_make_directories
from finrl.config import INDICATORS, TRAINED_MODEL_DIR, RESULTS_DIR
from pypfopt.efficient_frontier import EfficientFrontier
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
import matplotlib.pyplot as plt
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts
from finrl.config_tickers import SP_500_TICKER


check_and_make_directories([TRAINED_MODEL_DIR])

In [41]:
# train = pd.read_csv('data/train.csv')
# train = train.set_index(train.columns[0])
# train.index.names = ['']

# trade = pd.read_csv('data/trade.csv')
# trade = trade.set_index(trade.columns[0])
# trade.index.names = ['']

In [42]:

train = pd.read_pickle('data/train.pickle')
train.index.names = ['']

trade = pd.read_pickle('data/trade.pickle')
trade.index.names = ['']

In [43]:
trade.head()

,date,close,high,low,open,volume,tic,day,gdp_log,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence,daily_return
,,,,,,,,,,,,,,,,,,,
0,2022-07-01,116.428886,116.966058,113.977441,116.018687,1119800,A,4,10.178704,-1.243371,126.593236,106.836674,47.215873,-41.333927,3.151486,118.512245,119.273007,739.043765,1.591457
0,2022-07-01,13.130000,13.180000,12.520000,12.780000,34110400,AAL,4,10.178704,-0.929487,16.782486,11.131514,42.042263,-77.882730,26.682854,14.950667,16.521667,739.043765,-0.887227
0,2022-07-01,160.159088,163.866991,158.766325,159.937716,784400,AAP,4,10.178704,-3.802959,176.455017,150.658110,41.489762,-60.850795,18.128198,166.314553,180.640219,739.043765,11.197950
0,2022-07-01,136.581284,136.689425,133.366576,133.740142,71051600,AAPL,4,10.178704,-2.301431,147.113387,125.673665,45.052159,-36.439697,17.240171,138.153919,147.386978,739.043765,-0.147215
0,2022-07-01,103.027710,103.261396,100.625413,101.148868,3291700,ABT,4,10.178704,-1.119116,110.437225,92.840127,47.685119,-19.392710,4.354595,103.568614,105.811822,739.043765,-0.245667


In [44]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2 * stock_dimension + len(INDICATORS) * stock_dimension + 1 * stock_dimension
print(f'Stock Dimension: {stock_dimension}, state space: {state_space}')

Stock Dimension: 393, state space: 4324


In [45]:
df_account_value_dt = pd.read_pickle('data/df_account_value_dt.pkl')
df_actions_dt = pd.read_pickle('data/df_actions_dt.pkl')

In [46]:
df_account_value_ppo = pd.read_pickle('data/df_account_value_ppo.pkl')
df_actions_ppo = pd.read_pickle('data/df_actions_ppo.pkl')

In [47]:
# Helps us process data into a form for weight calculation
def process_df_for_mvo(df):
  df = df.sort_values(['date','tic'],ignore_index=True)[['date','tic','close']]
  fst = df
  fst = fst.iloc[0:stock_dimension, :]
  tic = fst['tic'].tolist()

  mvo = pd.DataFrame()

  for k in range(len(tic)):
    mvo[tic[k]] = 0

  for i in range(df.shape[0]//stock_dimension):
    n = df
    n = n.iloc[i * stock_dimension:(i+1) * stock_dimension, :]
    date = n['date'][i*stock_dimension]
    mvo.loc[date] = n['close'].tolist()

  return mvo

# Calculates weights of average return and covariance matrix
def StockReturnsComputing(StockPrice, Rows, Columns):
  StockReturn = np.zeros([Rows-1, Columns])
  for j in range(Columns):        # j: Assets
    for i in range(Rows-1):     # i: Daily Prices
      StockReturn[i,j]=((StockPrice[i+1, j]-StockPrice[i,j])/StockPrice[i,j])* 100

  return StockReturn

In [48]:
StockData = process_df_for_mvo(train)
TradeData = process_df_for_mvo(trade)

TradeData.to_numpy()

C:\Users\drebi\AppData\Local\Temp\ipykernel_53116\294320812.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mvo[tic[k]] = 0
C:\Users\drebi\AppData\Local\Temp\ipykernel_53116\294320812.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mvo[tic[k]] = 0
C:\Users\drebi\AppData\Local\Temp\ipykernel_53116\294320812.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) i

array([[116.429,  13.13 , 160.159, ..., 109.665, 104.35 ,  45.352],
       [117.747,  13.88 , 168.156, ..., 110.248, 103.98 ,  45.087],
       [118.353,  13.89 , 167.658, ..., 109.778, 102.852,  44.495],
       ...,
       [136.162,  13.88 ,  72.682, ..., 137.695, 117.707,  39.708],
       [137.991,  13.98 ,  72.953, ..., 139.074, 120.034,  39.29 ],
       [135.47 ,  13.51 ,  70.593, ..., 137.19 , 118.624,  38.776]],
      shape=(460, 393))

In [49]:
TradeData.head()

,A,AAL,AAP,AAPL,ABT,ACN,ADBE,ADI,ADM,ADP,...,WU,WY,WYNN,XEL,XOM,XRAY,XRX,YUM,ZBH,ZION
2022-07-01,116.428886,13.13,160.159088,136.581284,103.027710,265.930206,368.480011,134.889374,69.337593,199.049942,...,12.820278,30.298096,56.667641,65.121368,78.262825,33.328892,11.309069,109.665192,104.349701,45.351654
2022-07-05,117.747414,13.88,168.155975,139.166840,102.167755,260.737671,376.489990,135.040405,65.680588,198.349823,...,12.789774,30.494259,56.287785,62.664639,75.813477,33.236813,11.196939,110.247757,103.980034,45.086845
2022-07-06,118.352943,13.89,167.657944,140.503830,101.597549,261.467285,382.829987,136.578873,65.535416,202.811951,...,12.820278,30.253508,54.388470,63.668854,74.445786,32.794891,10.964670,109.777946,102.851639,44.495457
2022-07-07,120.276970,14.10,169.705566,143.875854,102.345337,264.745758,390.890015,140.231705,67.513626,202.121155,...,12.774519,30.743916,56.355961,62.386696,76.823608,32.647583,11.333095,110.633018,103.172630,45.439915
2022-07-08,119.651909,13.97,168.700165,144.554199,102.130348,264.433075,389.440002,141.524826,66.551735,202.821259,...,12.621987,31.216482,54.992352,62.404625,76.948761,32.417408,11.517309,110.200775,102.472229,45.051548


In [50]:
initial_portfolio_value = 1000000

assets = trade['tic'].unique()
n = len(assets)

naive_weights = {asset: 1/n for asset in assets}

naive_weights_df = pd.DataFrame.from_dict(naive_weights, orient='index', columns=['Weight'])

print("Naive Strategy Weights:")
print(naive_weights_df)

trade_with_weights = trade.merge(naive_weights_df, left_on='tic', right_index=True)

trade_with_weights['1/n'] = trade_with_weights['close'] * trade_with_weights['Weight'] * initial_portfolio_value

portfolio_value = trade_with_weights.groupby('date')['1/n'].sum()

naive_result = portfolio_value / portfolio_value.iloc[0] * initial_portfolio_value

print("Portfolio Value Using Naive Strategy:")
print(naive_result)

Naive Strategy Weights:
        Weight
A     0.002545
AAL   0.002545
AAP   0.002545
AAPL  0.002545
ABT   0.002545
...        ...
XRAY  0.002545
XRX   0.002545
YUM   0.002545
ZBH   0.002545
ZION  0.002545

[393 rows x 1 columns]
Portfolio Value Using Naive Strategy:
date
2022-07-01    1.000000e+06
2022-07-05    9.967905e+05
2022-07-06    9.999099e+05
2022-07-07    1.011284e+06
2022-07-08    1.008732e+06
                  ...     
2024-04-24    1.308151e+06
2024-04-25    1.303476e+06
2024-04-26    1.306624e+06
2024-04-29    1.313567e+06
2024-04-30    1.294015e+06
Name: 1/n, Length: 460, dtype: float64


In [51]:
#compute asset returns
arStockPrices = np.asarray(StockData)
[Rows, Cols]=arStockPrices.shape
arReturns = StockReturnsComputing(arStockPrices, Rows, Cols)

#compute mean returns and variance covariance matrix of returns
meanReturns = np.mean(arReturns, axis = 0)
covReturns = np.cov(arReturns, rowvar=False)

#set precision for printing results
np.set_printoptions(precision=3, suppress = True)

#display mean returns and variance-covariance matrix of returns
print('Mean returns of assets in k-portfolio 1\n', meanReturns)
print('Variance-Covariance matrix of returns\n', covReturns)

Mean returns of assets in k-portfolio 1
 [0.089 0.082 0.069 0.131 0.061 0.082 0.102 0.087 0.053 0.072 0.09  0.054
 0.054 0.058 0.064 0.083 0.089 0.077 0.077 0.081 0.1   0.139 0.086 0.064
 0.097 0.169 0.077 0.06  0.062 0.105 0.081 0.131 0.068 0.093 0.036 0.069
 0.086 0.057 0.066 0.066 0.073 0.077 0.089 0.092 0.067 0.072 0.04  0.03
 0.068 0.053 0.036 0.071 0.041 0.116 0.072 0.059 0.087 0.065 0.062 0.048
 0.038 0.052 0.047 0.07  0.06  0.126 0.087 0.022 0.129 0.098 0.103 0.07
 0.039 0.102 0.071 0.041 0.047 0.079 0.064 0.078 0.114 0.088 0.076 0.112
 0.054 0.078 0.099 0.06  0.082 0.035 0.095 0.115 0.052 0.086 0.101 0.078
 0.054 0.05  0.047 0.071 0.064 0.085 0.045 0.102 0.101 0.059 0.076 0.09
 0.073 0.079 0.066 0.05  0.049 0.046 0.067 0.081 0.077 0.058 0.05  0.074
 0.044 0.102 0.081 0.049 0.071 0.098 0.066 0.058 0.066 0.075 0.035 0.059
 0.109 0.03  0.049 0.101 0.115 0.083 0.076 0.085 0.06  0.024 0.081 0.068
 0.106 0.046 0.074 0.046 0.032 0.059 0.012 0.048 0.048 0.069 0.063 0.091
 0.061 0.073 

In [52]:
ef_mean = EfficientFrontier(meanReturns, covReturns, weight_bounds=(0, 0.5), solver='ECOS')
raw_weights_mean = ef_mean.max_sharpe()
cleaned_weights_mean = ef_mean.clean_weights()

# solver_options = {'verbose': True}
# raw_weights_mean = ef_mean.max_sharpe(solver='ECOS', solver_options=solver_options)
# Use shape of 'Last Price' in the range to avoid shape mismatch error
mvo_weights = np.array([1000000 * cleaned_weights_mean[i] for i in range(393)])

mvo_weights

array([     0.,      0.,      0., 118410.,      0.,      0.,      0.,
            0.,      0.,      0.,      0.,      0.,      0.,      0.,
            0.,      0.,      0.,      0.,      0.,      0.,      0.,
            0.,      0.,      0.,      0.,      0.,      0.,      0.,
            0.,      0.,      0.,  39510.,      0.,      0.,      0.,
            0.,      0.,      0.,      0.,      0.,      0.,  32340.,
            0.,  94940.,      0.,      0.,      0.,      0.,      0.,
            0.,      0.,      0.,      0.,      0.,      0.,      0.,
            0.,      0.,      0.,      0.,      0.,      0.,      0.,
            0.,      0.,      0.,      0.,      0.,      0.,      0.,
            0.,  47680.,      0.,      0.,      0.,      0.,      0.,
            0.,      0.,      0.,  18120.,      0.,  84140.,      0.,
            0.,      0.,      0.,      0.,      0.,      0.,      0.,
            0.,      0.,      0.,      0.,      0.,      0.,      0.,
            0.,     

In [53]:
mvo_weights.shape

(393,)

In [54]:
LastPrice = np.array([1/p for p in StockData.tail(1).to_numpy()[0]])

In [55]:
LastPrice.shape

(393,)

In [56]:
Initial_Portfolio = np.multiply(mvo_weights, LastPrice)

Initial_Portfolio

array([   0.   ,    0.   ,    0.   ,  880.97 ,    0.   ,    0.   ,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.   ,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.   ,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.   ,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.   ,
          0.   ,  371.999,    0.   ,    0.   ,    0.   ,    0.   ,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,  232.929,
          0.   ,   44.176,    0.   ,    0.   ,    0.   ,    0.   ,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.   ,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.   ,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.   ,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,  534.848,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.   ,
          0.   ,    0.   ,  693.053,    0.   , 1382.262,    0.   ,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.

In [57]:
Portfolio_Assets = TradeData @ Initial_Portfolio
MVO_result = pd.DataFrame(Portfolio_Assets, columns=["Mean Var"])

MVO_result

,Mean Var
2022-07-01,1.012667e+06
2022-07-05,1.013432e+06
2022-07-06,1.020750e+06
2022-07-07,1.029825e+06
2022-07-08,1.028529e+06
...,...
2024-04-24,1.409321e+06
2024-04-25,1.413204e+06
2024-04-26,1.414839e+06
2024-04-29,1.425924e+06


In [58]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2022-07-01'
TRADE_START_DATE = '2022-07-01'
TRADE_END_DATE = '2024-05-01'


In [59]:
df_sp = YahooDownloader(start_date = TRADE_START_DATE,
                        end_date = TRADE_END_DATE,
                        ticker_list = ['^GSPC']).fetch_data()

[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (460, 8)


In [60]:
df_sp = df_sp[['date','close']]
fst_day = df_sp['close'][0]
sp = pd.merge(df_sp['date'], df_sp['close'].div(fst_day).mul(1000000),
               how='outer', left_index=True, right_index=True).set_index('date')

In [61]:
sp

,close
date,
2022-07-01,1.000000e+06
2022-07-05,1.001584e+06
2022-07-06,1.005163e+06
2022-07-07,1.020205e+06
2022-07-08,1.019358e+06
...,...
2024-04-24,1.325802e+06
2024-04-25,1.319734e+06
2024-04-26,1.333208e+06


In [70]:
df_result_dt = df_account_value_dt.set_index(df_account_value_dt.columns[0]) #if if_using_a2c else None

result = pd.DataFrame()
dt_result = pd.merge(result, df_result_dt, how='outer', left_index=True, right_index=True)

col_name = []
col_name.append('DT')
dt_result.columns = col_name
dt_result

,DT
date,
2022-07-01,1.000000e+06
2022-07-05,9.974330e+05
2022-07-06,9.986263e+05
2022-07-07,1.005778e+06
2022-07-08,1.003415e+06
...,...
2024-04-24,1.327640e+06
2024-04-25,1.323187e+06
2024-04-26,1.324050e+06


In [75]:
df_result_ppo = df_account_value_ppo.set_index(df_account_value_ppo.columns[0]) #if if_using_a2c else None
result = pd.DataFrame()
ppo_result = pd.merge(result, df_result_ppo, how='outer', left_index=True, right_index=True)

col_name = []
col_name.append('PPO')
ppo_result.columns = col_name
ppo_result

,PPO
date,
2022-07-01,1.000000e+06
2022-07-05,9.975521e+05
2022-07-06,9.986649e+05
2022-07-07,1.005271e+06
2022-07-08,1.002485e+06
...,...
2024-04-24,1.380024e+06
2024-04-25,1.376283e+06
2024-04-26,1.378705e+06


In [ ]:
dt_ppo = pd.merge(dt_result, ppo_result, how='outer', left_index=True, right_index=True)

,DT,PPO
date,,
2022-07-01,1.000000e+06,1.000000e+06
2022-07-05,9.974330e+05,9.975521e+05
2022-07-06,9.986263e+05,9.986649e+05
2022-07-07,1.005778e+06,1.005271e+06
2022-07-08,1.003415e+06,1.002485e+06
...,...,...
2024-04-24,1.327640e+06,1.380024e+06
2024-04-25,1.323187e+06,1.376283e+06
2024-04-26,1.324050e+06,1.378705e+06


In [ ]:
dt_ppo_mvo = pd.merge(dt_ppo, MVO_result, how='outer', left_index=True, right_index=True)
dt_ppo_mvo.index = dt_ppo_mvo.index.astype(str)

,DT,PPO,Mean Var
date,,,
2022-07-01,1.000000e+06,1.000000e+06,1.012667e+06
2022-07-05,9.974330e+05,9.975521e+05,1.013432e+06
2022-07-06,9.986263e+05,9.986649e+05,1.020750e+06
2022-07-07,1.005778e+06,1.005271e+06,1.029825e+06
2022-07-08,1.003415e+06,1.002485e+06,1.028529e+06
...,...,...,...
2024-04-24,1.327640e+06,1.380024e+06,1.409321e+06
2024-04-25,1.323187e+06,1.376283e+06,1.413204e+06
2024-04-26,1.324050e+06,1.378705e+06,1.414839e+06


In [94]:
dt_ppo_mvo_sp= pd.merge(dt_ppo_mvo, sp, how='outer', left_index=True, right_index=True)
dt_ppo_mvo_sp

,DT,PPO,Mean Var,close
date,,,,
2022-07-01,1.000000e+06,1.000000e+06,1.012667e+06,1.000000e+06
2022-07-05,9.974330e+05,9.975521e+05,1.013432e+06,1.001584e+06
2022-07-06,9.986263e+05,9.986649e+05,1.020750e+06,1.005163e+06
2022-07-07,1.005778e+06,1.005271e+06,1.029825e+06,1.020205e+06
2022-07-08,1.003415e+06,1.002485e+06,1.028529e+06,1.019358e+06
...,...,...,...,...
2024-04-24,1.327640e+06,1.380024e+06,1.409321e+06,1.325802e+06
2024-04-25,1.323187e+06,1.376283e+06,1.413204e+06,1.319734e+06
2024-04-26,1.324050e+06,1.378705e+06,1.414839e+06,1.333208e+06


pandas.core.indexes.datetimes.DatetimeIndex

In [99]:
naive_result.index = naive_result.index.astype(str)
final_result = pd.merge(dt_ppo_mvo_sp, naive_result, how='outer', left_index=True, right_index=True)


In [100]:
final_result.rename(columns={'close':'S&P500'}, inplace=True)


In [101]:
final_result

,DT,PPO,Mean Var,S&P500,1/n
date,,,,,
2022-07-01,1.000000e+06,1.000000e+06,1.012667e+06,1.000000e+06,1.000000e+06
2022-07-05,9.974330e+05,9.975521e+05,1.013432e+06,1.001584e+06,9.967905e+05
2022-07-06,9.986263e+05,9.986649e+05,1.020750e+06,1.005163e+06,9.999099e+05
2022-07-07,1.005778e+06,1.005271e+06,1.029825e+06,1.020205e+06,1.011284e+06
2022-07-08,1.003415e+06,1.002485e+06,1.028529e+06,1.019358e+06,1.008732e+06
...,...,...,...,...,...
2024-04-24,1.327640e+06,1.380024e+06,1.409321e+06,1.325802e+06,1.308151e+06
2024-04-25,1.323187e+06,1.376283e+06,1.413204e+06,1.319734e+06,1.303476e+06
2024-04-26,1.324050e+06,1.378705e+06,1.414839e+06,1.333208e+06,1.306624e+06


In [102]:
plt.rcParams["figure.figsize"] = (15,5)
ax = result[['DT', '1/n']].plot()
plt.title("DT vs Naive Account Value Over Time")
plt.xlabel("date")
plt.ylabel("Account Value")
plt.legend(["DT", "Naive"])
plt.grid(True)
plt.savefig('DT vs Naive Account Value Over Time.png')


KeyError: "None of [Index(['DT', '1/n'], dtype='object')] are in the [columns]"

In [ ]:
plt.rcParams['figure.figsize'] = (15,5)
ax = result[['PPO', '1/n']].plot()
plt.title('PPO vs Naive Account Value Over Time')
plt.xlabel('date')
plt.ylabel('Account Value')
plt.legend(['PPO', 'Naive'])
plt.grid(True)
plt.savefig('PPO vs Naive Account Value Over Time.png')



In [ ]:
plt.rcParams['figure.figsize'] = (15, 5)
ax = result[['PPO', 'DT']].plot()
plt.title('PPO vs DT Account Value Over Time')
plt.xlabel('date')
plt.ylabel('Account Value')
plt.legend(['PPO', 'DT'])
plt.grid(True)
plt.savefig('PPO vs DT Account Value Over Time.png')


In [ ]:
plt.rcParams['figure.figsize'] = (15, 5)
ax = result[['DT', '1/n', 'PPO']].plot()
plt.title('DT, Naive, and PPO Account Value Over Time')
plt.xlabel('date')
plt.ylabel('Account Value')
plt.legend(['DT', 'Naive', 'PPO'])
plt.grid(True)
plt.savefig('DT, Naive, and PPO Account Value Over Time.png')





In [103]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 2, figsize=(18, 12))

# First plot: DT vs Naive
# DT = blue, PPO = red, Naive = green
result[['DT', '1/n']].plot(ax=axs[0, 0], color=['blue', 'green'])
axs[0, 0].set_xlabel('Date', fontsize=16)
axs[0, 0].set_ylabel('Account Value', fontsize=16)
axs[0, 0].legend(['DT', 'Naive'], fontsize=14)
axs[0, 0].grid(True)

# Second plot: PPO vs Naive
result[['PPO', '1/n']].plot(ax=axs[0, 1], color=['red', 'green'])
axs[0, 1].set_xlabel('Date', fontsize=16)
axs[0, 1].set_ylabel('Account Value', fontsize=16)
axs[0, 1].legend(['PPO', 'Naive'], fontsize=14)
axs[0, 1].grid(True)

# Third plot: PPO vs DT
result[['PPO', 'DT']].plot(ax=axs[1, 0], color=['red', 'blue'])
axs[1, 0].set_xlabel('Date', fontsize=16)
axs[1, 0].set_ylabel('Account Value', fontsize=16)
axs[1, 0].legend(['PPO', 'DT'], fontsize=14)
axs[1, 0].grid(True)

# Fourth plot: DT, Naive, and PPO
result[['DT', '1/n', 'PPO']].plot(ax=axs[1, 1], color=['blue', 'green', 'red'])
axs[1, 1].set_xlabel('Date', fontsize=16)
axs[1, 1].set_ylabel('Account Value', fontsize=16)
axs[1, 1].legend(['DT', 'Naive', 'PPO'], fontsize=14)
axs[1, 1].grid(True)

plt.tight_layout()
plt.savefig('plots/results.png')
plt.show()


KeyError: "None of [Index(['DT', '1/n'], dtype='object')] are in the [columns]"

In [36]:
from pyfolio import timeseries
DLR_strat = convert_daily_return_to_pyfolio_ts(result['DT'])
perf_func = timeseries.perf_stats
perf_stats_all = perf_func(returns=DLR_strat,
                       factor_returns=DLR_strat,
                        positions=None,
                        transactions=None,
                        turnover_denom="AGB")

KeyError: 'date'

In [104]:
df_daily_return_dt = final_result[['DT']]
df_daily_return_dt = df_daily_return_dt.reset_index()
df_daily_return_dt['daily_return'] = df_daily_return_dt['DT'].pct_change()
df_daily_return_dt.drop(columns=['DT'], inplace=True)
df_daily_return_dt.head()

DLR_strat = convert_daily_return_to_pyfolio_ts(df_daily_return_dt)
perf_func = timeseries.perf_stats

perf_stats_all = perf_func(returns=DLR_strat,
                           factor_returns=DLR_strat,
                           positions=None,
                           transactions=None,
                           turnover_denom='AGB')

perf_stats_all

Annual return          0.160969
Cumulative returns     0.313177
Annual volatility      0.150141
Sharpe ratio           1.071349
Calmar ratio           1.107778
Stability              0.801751
Max drawdown          -0.145308
Omega ratio            1.198363
Sortino ratio          1.624205
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.071242
Daily value at risk   -0.018278
Alpha                  0.000000
Beta                   1.000000
dtype: float64

In [105]:
df_daily_return_naive = final_result[['1/n']]
df_daily_return_naive = df_daily_return_naive.reset_index()
df_daily_return_naive['daily_return'] = df_daily_return_naive['1/n'].pct_change()
df_daily_return_naive.drop(columns=['1/n'], inplace=True)
df_daily_return_naive.head()

DLR_strat = convert_daily_return_to_pyfolio_ts(df_daily_return_naive)
perf_func = timeseries.perf_stats

perf_stats_all = perf_func(returns=DLR_strat,
                           factor_returns=DLR_strat,
                           positions=None,
                           transactions=None,
                           turnover_denom='AGB')

perf_stats_all

Annual return          0.151657
Cumulative returns     0.294015
Annual volatility      0.156061
Sharpe ratio           0.984721
Calmar ratio           0.975996
Stability              0.769817
Max drawdown          -0.155387
Omega ratio            1.180527
Sortino ratio          1.498265
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.093011
Daily value at risk   -0.019052
Alpha                  0.000000
Beta                   1.000000
dtype: float64

In [106]:
df_daily_return_ppo = final_result[['PPO']]
df_daily_return_ppo = df_daily_return_ppo.reset_index()
df_daily_return_ppo['daily_return'] = df_daily_return_ppo['PPO'].pct_change()
df_daily_return_ppo.drop(columns=['PPO'], inplace=True)
df_daily_return_ppo.head()

DLR_strat = convert_daily_return_to_pyfolio_ts(df_daily_return_ppo)
perf_func = timeseries.perf_stats

perf_stats_all = perf_func(returns=DLR_strat,
                           factor_returns=DLR_strat,
                           positions=None,
                           transactions=None,
                           turnover_denom='AGB')

perf_stats_all

Annual return          0.187503
Cumulative returns     0.368479
Annual volatility      0.139969
Sharpe ratio           1.300689
Calmar ratio           1.531163
Stability              0.876133
Max drawdown          -0.122458
Omega ratio            1.237453
Sortino ratio          1.959002
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.122165
Daily value at risk   -0.016912
Alpha                  0.000000
Beta                   1.000000
dtype: float64